# Weather Forecasting Using Deep Learning

This project demonstrates how to forecast weather (temperature) using LSTM networks in Python. Synthetic data is used for demonstration purposes.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
def generate_temperature_data(num_days=365*2):
    time = np.arange(0, num_days, 1)
    temperature = 25 * np.sin(2 * np.pi * time / 365) + 15 + np.random.normal(0, 2, num_days)
    df = pd.DataFrame(temperature, columns=['temperature'])
    return df

df = generate_temperature_data()
df.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['temperature'].values.reshape(-1, 1))

def create_sequences(data, look_back=7):
    X, y = [], []
    for i in range(len(data) - look_back - 1):
        a = data[i:(i + look_back), 0]
        X.append(a)
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 7
X, y = create_sequences(scaled_data, look_back)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=2, validation_data=(X_test, y_test))

In [ ]:
def predict_next_day_temp(last_days_temp):
    scaled_input = scaler.transform(np.array(last_days_temp).reshape(-1, 1))
    input_seq = np.reshape(scaled_input, (1, look_back, 1))
    prediction = model.predict(input_seq, verbose=0)
    predicted_temp = scaler.inverse_transform(prediction)[0][0]
    return predicted_temp

last_days = df['temperature'].values[-look_back:]
predicted_temperature = predict_next_day_temp(last_days)
print(f"Last {look_back} days temperatures: {np.round(last_days, 2)}")
print(f"Predicted temperature for the next day: {predicted_temperature:.2f} °C")

This concludes the basic implementation of weather forecasting using LSTM and synthetic temperature data.